# DAV Lab 4 Homework
- Name:
- Roll No:

# Sentiment Analysis

Lets build a sentiment Analysis model, which can detect given a phrase/sentence, if its positive or negative

In [14]:
## Import Libraries
import numpy as np
import pandas as pd
import urllib.request
import nltk
import random
from nltk.corpus import stopwords
from nltk.corpus import movie_reviews
from nltk.tokenize import word_tokenize

## Dataset
Sentiment Analysis is the process of computationally identifying and categorizing opinions expressed in a piece of text, especially in order to determine whether the writer’s attitude towards a particular topic, product, etc. is positive, negative, or neutral.

This is a simple project of classifying the movie reviews as either positive or negative. We would be working on the ‘movie_reviews’ dataset in ntlk.corpus package

## Tasks
### EDA
1. Check the size of the corpus using `len()`. This will give you the total number of reviews in the corpus.
1. Check the categories in the corpus using categories(). This will give you a list of the two categories("neg" and "pos").Which indicate whether a review is negative or positive.
1. Count the number of positive and negative reviews using a loop and `fileids()`. This will give you an idea of the distribution of reviews in the corpus.
1. View a specific review using `raw()` and `fileids()`. This will allow you to read a review in its raw text form.
1. Perform tokenization using `word_tokenize()` or `sent_tokenize()`. This will allow you to see how the reviews are broken down into individual words or sentences.
1. Perform part-of-speech (POS) tagging using `pos_tag()`. This will allow you to see how different parts of speech are used in the reviews.
1. Create a list of all reviews in the corpus, with their respective categories.
## Model Building
1. Create a list of stop words using `stopwords.words('english')` and remove them from the text using a list comprehension. This will remove common words like "the", "and", and "a" that do not add much meaning to the text.
1. Convert the words to lowercase using `lower()`. This will ensure that words with different capitalizations are treated as the same word.
1. Use `FreqDist()` to create a frequency distribution of the most common words in the corpus. You can specify the number of most common words to include using the `most_common()` method.
1. Create a feature set from the most common words by creating a list of tuples where each tuple contains a dictionary of features and the corresponding category of the review (i.e., "neg" or "pos"). The dictionary of features should contain the most common words as keys and their frequencies in each review as values.
1. Shuffle the feature set using `random.shuffle()` to ensure that the training and test sets are representative of the overall corpus.
1. Split the feature set into a training set and a test set using a 80/20 split. This will allow you to train the classifier on a portion of the data and test its accuracy on the remaining data.
1. Train a naive Bayes classifier using `nltk.classify.NaiveBayesClassifier.train()` and the training set of features.
1. Test the accuracy of the classifier on the test set using `nltk.classify.accuracy()`.
1. Show the most informative features using `classifier.show_most_informative_features(10)`
1. Experiment with different numbers of most common words and different thresholds for including words in the feature set to see how they affect the accuracy of the classifier.

## References
1. https://www.nltk.org/howto/sentiment.html
2. https://www.analyticsvidhya.com/blog/2021/06/sentiment-analysis-using-nltk-a-practical-approach/

In [15]:
nltk.download([ "stopwords", "movie_reviews", "punkt"])

[nltk_data] <urlopen error [Errno 104] Connection reset by peer>


False

In [10]:
# Prints total number of words in 'movie_reviews'

# Print the review categories

# Prints all file ids

# Prints file ids of positive reviews

# Prints file ids of negative reviews.